# Exploration

## Data Cleaning 

#### Summary of what needs to be done to clean the data for each feature

##### Credits dataset

| Feature | Action | Description |
| -------- | ------------|-------------|
| movie_id | Dropped | Dropped during the merging of datasets |
| title | Dropped | Used the title feature from movies dataset |
| cast | Transformed | A list of the cast member names |
| crew | Transformed | A list of the crew member names |


#### Movies dataset
| Feature | Action | Description |
| -------- | ------------|-------------|
| budget | NA | NA |
| genres | Transformed | A list of genres of the film |
| homepage | Dropped! | Not needed for this version of project |
| id | NA | NA |
| keywords | Transformed | A list of keywords of the film |
| original_language | NA | NA |
| original_title |
| overview |
| popularity |
| production_companies |
| production_countries |
| release_date |
| revenue |
| runtime |
| spoken_languages |
| status |
| tagline |
| title |
| vote_average |
| vote_count |

data cleaning reminder -- <strong>Data Quality assessment</strong>

| Category | Completion? | Description |
| -------- | ------------|-------------|
| Accuracy | [ ] | Does it reflect reality over time |
| Completeness | [ ] | is the requisite information there |
| Consistency | [ x ] | Data match with other datasets |
| Currency | [ ] | Values up to date |
| Relevancy | [ ] | Relevant to objective |
| Validity | [ ] | Data containing valuable values |
| No duplicates | [ ] | Only one instance of a data point |

In [ ]:
from src.config import RAW_DATA_CREDITS,RAW_DATA_MOVIES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
df_credits = pd.read_csv(RAW_DATA_CREDITS)
df_movies = pd.read_csv(RAW_DATA_MOVIES)

In [ ]:
# functions to clean
def format_json_inputs(dataframe):
    """change json inputs into a list datatype

    Args:
        columns (list): _description_
        dataframe (pd.Dataframe): _description_
    """
    json_columns = ["genres","keywords","cast","crew","production_companies","production_countries","spoken_languages"]
    for column in json_columns:
        dataframe[column] = dataframe[column].apply(json.loads)
def get_info_from_dict_list(dataframe):
    #not include
    json_columns = ["genres","keywords","production_companies","production_countries","spoken_languages"]
    for column in json_columns:
        dataframe[column] = dataframe[column].apply(lambda x: [ _['name'] for _ in x])
def clean_status_then_removal(dataframe):
    post_production_index = dataframe[dataframe.status=='Post Production'].index
    rumored_index = dataframe[dataframe.status=='Rumored'].index

    combined_index = post_production_index.append(rumored_index)
    dataframe = dataframe.drop(combined_index,axis=0)
    dataframe = dataframe.drop('status',axis=1)
    return dataframe
def drop_columns(dataframe):
    drop_columns = ["homepage","original_title","title_y"]
    dataframe.drop(drop_columns,axis=1,inplace=True)
def string_to_datetime(dataframe,column):
    dataframe[column] = [pd.to_datetime(entity,format="%Y-%m-%d") for entity in dataframe[column]]